In [1]:
!python TransE.py

Epoch 1 | loss: 6.219164
Epoch 2 | loss: 0.000000
Epoch 3 | loss: 0.000000
Epoch 4 | loss: 0.000000
Epoch 5 | loss: 0.000000
Epoch 6 | loss: 0.000000
Epoch 7 | loss: 0.000000
Epoch 8 | loss: 0.000000
Epoch 9 | loss: 0.000000
Epoch 10 | loss: 0.000000
Epoch 11 | loss: 0.000000
Epoch 12 | loss: 0.000000
Epoch 13 | loss: 0.000000
Epoch 14 | loss: 0.000000
Epoch 15 | loss: 0.000000
Epoch 16 | loss: 0.000000
Epoch 17 | loss: 0.012650
Epoch 18 | loss: 0.000000
Epoch 19 | loss: 0.000094
Epoch 20 | loss: 0.015199
Epoch 21 | loss: 0.000000
Epoch 22 | loss: 0.000000
Epoch 23 | loss: 0.000000
Epoch 24 | loss: 0.000000
Epoch 25 | loss: 0.000000
Epoch 26 | loss: 0.000000
Epoch 27 | loss: 0.000000
Epoch 28 | loss: 0.000000
Epoch 29 | loss: 0.000000
Epoch 30 | loss: 0.000000
Epoch 31 | loss: 0.000000
Epoch 32 | loss: 0.000000
Epoch 33 | loss: 0.000000
Epoch 34 | loss: 0.000000
Epoch 35 | loss: 0.000000
Epoch 36 | loss: 0.000000
Epoch 37 | loss: 0.000000
Epoch 38 | loss: 0.000000
Epoch 39 | loss: 0.00

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import time
import numpy as np
from TransE import TransE
from TransE import Config
from load_data import PyTorchTrainDataLoader

In [2]:
#初始化模型
config = Config()
train_dataloader = PyTorchTrainDataLoader(
                            in_path = "./data/", 
                            nbatches = config.nbatches,
                            threads = 8)
    
transe = TransE(
            ent_tot = train_dataloader.get_ent_tot(),
            rel_tot = train_dataloader.get_rel_tot(),
            dim = config.hidden_size, #50
            p_norm = config.p_norm, 
            norm_flag = True,
            margin=config.margin)

In [3]:
#初始化wikidata词条与embedding序号之间的字典
ent_dic = {}
rel_dic = {}
f = open('./data/entity2id.txt','r')
next(f)
for index in range(train_dataloader.get_ent_tot()):
    value,key = f.readline().strip().split()
    ent_dic[int(key)] = value
f = open('./data/relation2id.txt','r')
next(f)
for index in range(train_dataloader.get_rel_tot()):
    value,key = f.readline().strip().split()
    rel_dic[int(key)] = value

In [4]:
#载入预训练的embedding参数
ent_data = np.loadtxt('entity2vec_margin1.txt')
rel_data = np.loadtxt('relation2vec_margin1.txt')
ent_data = torch.Tensor(ent_data)
rel_data = torch.Tensor(rel_data)
transe.ent_embeddings = transe.ent_embeddings.from_pretrained(ent_data)
transe.rel_embeddings = transe.rel_embeddings.from_pretrained(rel_data)

In [5]:
transe.ent_embeddings.weight

Parameter containing:
tensor([[ 1.4980e-03,  7.3000e-05,  1.3100e-04,  ...,  4.2400e-04,
          1.6300e-04,  3.2700e-04],
        [ 9.4770e-03, -6.1700e-04, -1.0322e-02,  ...,  9.7870e-03,
          8.3210e-03,  5.0400e-03],
        [-2.8906e-02, -9.3000e-05, -3.7026e-02,  ...,  2.1100e-04,
          4.8500e-04, -1.3220e-03],
        ...,
        [-1.3000e-05,  1.5520e-03,  3.2000e-05,  ..., -1.1334e-02,
         -1.6550e-03,  2.2000e-05],
        [-2.7000e-05,  1.4000e-05, -4.9370e-03,  ...,  1.9000e-05,
          6.7660e-03,  1.0450e-02],
        [ 8.7750e-03, -8.1330e-03, -6.9990e-03,  ...,  6.6280e-03,
         -0.0000e+00, -9.4220e-03]])

In [6]:
transe.rel_embeddings.weight

Parameter containing:
tensor([[-6.4514e-02,  1.1022e-01,  3.7873e-02,  ...,  4.0954e-02,
          1.7540e-03,  4.6698e-02],
        [ 4.7767e-02,  9.0294e-02, -1.2000e-05,  ...,  5.2598e-02,
          6.3662e-02, -2.8098e-02],
        [ 2.4515e-02, -1.8321e-02,  1.5092e-02,  ...,  4.1269e-02,
         -4.6200e-04,  1.3514e-02],
        ...,
        [-3.6629e-02, -7.1896e-02,  2.6840e-02,  ..., -6.6022e-02,
         -4.3655e-02, -1.1271e-01],
        [ 7.8148e-02, -3.2420e-02,  5.9684e-02,  ..., -1.1181e-01,
          5.3526e-02,  2.9649e-02],
        [-1.5445e-02,  2.4218e-02, -2.9010e-03,  ...,  4.7367e-02,
          1.0299e-02,  5.2378e-02]])

In [7]:
#预测Q30+P36最接近的尾实体
data = {'batch_h':torch.LongTensor([list(ent_dic.keys())[list(ent_dic.values()).index('Q30')]]),
        'batch_r':torch.LongTensor([list(rel_dic.keys())[list(rel_dic.values()).index('P36')]]),
        'batch_t':torch.LongTensor([i for i in range(train_dataloader.get_ent_tot())])}

In [8]:
score = transe.predict(data)
for index in score.argsort()[0:10]:
    print(ent_dic[index])

Q20
Q34266
Q29
Q33946
Q41304
Q33
Q35
Q403
Q148
Q34


In [9]:
score_WashingtonDC = score[list(ent_dic.keys())[list(ent_dic.values()).index('Q61')]]
score_WashingtonDC

1.6899345

In [10]:
per = (score_WashingtonDC - score.min())/(score.max()-score.min())
per

0.44240236

In [11]:
data_1 = {'batch_h':torch.LongTensor([list(ent_dic.keys())[list(ent_dic.values()).index('Q30')]]),
        'batch_t':torch.LongTensor([list(ent_dic.keys())[list(ent_dic.values()).index('Q49')]]),
        'batch_r':torch.LongTensor([i for i in range(train_dataloader.get_rel_tot())])}
score_1 = transe.predict(data_1)

In [12]:
for index in score_1.argsort()[0:10]:
    print(rel_dic[index])

P30
P741
P58
P790
P927
P123
P2321
P749
P186
P750
